# Interpretability analysis CIFAR10

## Installation of needed libraries

In [ ]:
from IPython.display import clear_output
!pip install build
try:
    import pytorch_trainers_interpretability
except:
    !cd .. && python -m build
    clear_output()
    !pip install --force --upgrade ../dist/pytorch_trainers_interpretability-0.0.1-py3-none-any.whl
    !pip install torchmetrics[image] tqdm
clear_output()
try:
    import pytorch_trainers_interpretability
except:
    raise Exception("Packages not installed! Please try again!")
print("Installation completed!")

## Standard and Adversarial training Small ImageNet 150

In [2]:
import torch
from pytorch_trainers_interpretability.trainers import StandardTrainer, AdversarialTrainer
from pytorch_trainers_interpretability.attack import L2Step
from pytorch_trainers_interpretability.models.resnet  import ResNet50
from torchvision import datasets, transforms
import json
import os
import tarfile

In [ ]:
if not os.path.isdir("./smallimagenet"):
    !python gdownload.py "1_nc-VVObGiOoS73gEH-UhVUsIytU0YRK" "smallimagenet.tar.gz"
    file = tarfile.open('smallimagenet.tar.gz')
    file.extractall('./')
    file.close()
    os.remove('./smallimagenet.tar.gz')
if not os.path.isfile("./Small ImageNet 150_labels.json"):
    !python gdownload.py "1t71KG_u-X-LCAFJ94Kg0pqNBajumEEsu" "Small ImageNet 150_labels.json"

In [ ]:
traindir = os.path.join("./smallimagenet", 'train')
valdir = os.path.join("./smallimagenet", 'test')
normalize = transforms.Normalize(mean=[0.4808, 0.4512, 0.4072],
                                     std=[0.2687, 0.2610, 0.2742])
trainset = datasets.ImageFolder(traindir)
transform_train =   transforms.Compose([
          transforms.Resize(140),
          transforms.RandomResizedCrop(128),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
    ])
transform_test = transforms.Compose([
          transforms.Resize(140),
          transforms.CenterCrop(128),
          transforms.ToTensor(),
    ])
testset = datasets.ImageFolder(valdir, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                         shuffle=True)  
f = open('./Small ImageNet 150_labels.json')

classes = json.load(f)

In [ ]:
model = ResNet50(num_classes=150)
training_kwargs = {
    "model": model,
    "pretrained": False,
    "batch_size": 200,
    "adv_step": L2Step,
    "adv_epsilon": 0.5,
    "adv_iter": 20,
    "adv_lr": 0.1,
    "lr": 0.0001,
    "epochs": 100,
    "optimizer": "Adam",
    "weight_decay": 5e-4,
    "lr_scheduler": "CosineAnnealingLR",
    "testset": testset,
    "trainset": trainset,
    "transforms_train": transform_train,
    "transforms_test": transform_test,
    "input_normalizer": normalize,
    "save_path": "./smimagenet_standard_model"
}
trainer = StandardTrainer(**training_kwargs)
trainer()
trainer.eval()
trainer.eval_adv()

In [ ]:
model2 = ResNet50(num_classes=150)
training_kwargs2 = {
    "model": model2,
    "pretrained": False,
    "batch_size": 200,
    "adv_step": L2Step,
    "adv_epsilon": 0.5,
    "adv_iter": 20,
    "adv_lr": 0.1,
    "lr": 0.0001,
    "epochs": 100,
    "optimizer": "Adam",
    "weight_decay": 5e-4,
    "lr_scheduler": "CosineAnnealingLR",
    "testset": testset,
    "trainset": trainset,
    "transforms_train": transform_train,
    "transforms_test": transform_test,
    "input_normalizer": normalize,
    "save_path": "./smimagenet_robust_l2_model"
}
trainer2 = AdversarialTrainer(**training_kwargs2)
trainer2()
trainer2.eval_nat()
trainer2.eval_adv()